# 線形MDPでの切り替えコスト

参考：
* [A Provably Efficient Algorithm for Linear Markov Decision Process with Low Switching Cost](https://arxiv.org/abs/2101.00494)


[RL_switching_cost_model-free](RL_switching_cost_model-free.ipynb)ではlocalな切り替えコストについて学びました．
$$
N_{\mathrm{switch}}^{\mathrm{loc}} \triangleq \sum_{k=1}^{K-1}\left|\left\{(h, x) \in[H] \times \mathcal{S}: \pi_k^h(x) \neq \pi_{k+1}^h(x)\right\}\right|
$$
これは状態空間のサイズに依存する点で，大規模な問題ではあまり好ましくありません．今回はもっと自然なGlobalな切り替えコストについて考えます：
$$
N_{\text {switch }}^{\mathrm{gl}} \triangleq \sum_{k=1}^{K-1} \mathbb{I}\left\{\pi_k \neq \pi_{k+1}\right\}
$$

明らかに
$$
N_{\text {switch }}^{\mathrm{gl}} \leq N_{\text {switch }}^{\mathrm{loc}} \leq|\mathcal{S}| H N_{\text {switch }}^{\mathrm{gl}}
$$
が成立しますね．

## 切り替えコストの下界

**注意**：多分[Sample-Efficient Reinforcement Learning with loglog(T) Switching Cost](https://arxiv.org/abs/2202.06385)の結果のほうがいい．$\sqrt{T}$リグレットが欲しい場合は$\log\log(T)$の切り替えコストは回避できないらしい．

TODO: 証明

## Low-切り替えコストなLinear-MDPアルゴリズム

基本的には普通の[Linear MDP](RL_General_linearMDP-Model-based.ipynb)と同じです．
次の更新を考えます：
$$
\begin{aligned}
& \Lambda_h^k \leftarrow \sum_{\tau=1}^{k-1} \boldsymbol{\phi}\left(x_h^\tau, a_h^\tau\right) \boldsymbol{\phi}\left(x_h^\tau, a_h^\tau\right)^{\top}+\lambda \cdot \mathbf{I} \\
& \mathbf{w}_h^k \leftarrow\left(\Lambda_h^k\right)^{-1} \sum_{\tau=1}^{k-1} \boldsymbol{\phi}\left(x_h^\tau, a_h^\tau\right)\left[r_h\left(x_h^\tau, a_h^\tau\right)+\max _a \widetilde{Q}_{h+1}^k\left(x_{h+1}^\tau, a\right)\right] \\
& \widetilde{Q}_h^k(\cdot, \cdot) \leftarrow \min \left\{\left(\mathbf{w}_h^k\right)^{\top} \boldsymbol{\phi}(\cdot, \cdot)+\beta\left[\boldsymbol{\phi}(\cdot, \cdot)^{\top}\left(\Lambda_h^k\right)^{-1} \boldsymbol{\phi}(\cdot, \cdot)\right]^{1 / 2}, H\right\}
\end{aligned}
$$

ただし，次のように方策の更新を制御します：

* もし$\left(\Lambda_h^{\tilde{k}}\right)^{-1} \npreceq 2\left(\Lambda_h^k\right)^{-1}$ならば，$\tilde{k} \leftarrow k$
    * ここで，$A \preceq B$は$B-A$が半正定値行列であることを意味します．
* そうでないなら$Q_h^k \leftarrow{\widetilde{Q}}_h^k$
* $Q_h^k$で探索を行う

---

**直感**

$\Lambda_h^k \leftarrow \sum_{\tau=1}^{k-1} \boldsymbol{\phi}\left(x_h^\tau, a_h^\tau\right) \boldsymbol{\phi}\left(x_h^\tau, a_h^\tau\right)^{\top}+\lambda \cdot \mathbf{I}$
が「今まで訪問した特徴ベクトルのカウント」を表していることを思い出しましょう（One-hotベクトルを考えればわかります）．

また，特徴ベクトル$\phi$に対して，$\Lambda^{-1} \phi$は$\frac{1}{\phi を訪問した回数}$を取り出す行為とほぼ同じです．つまり，$\phi$を訪問した回数が多いほど$\Lambda^{-1} \phi$は小さくなります．

さて，$2\Lambda^{-1} - \widetilde{\Lambda}^{-1}$の固有値$\lambda$とその固有ベクトル$\phi$は

$$
\left(\frac{2}{\phi \text{の訪問回数 at }\Lambda } - \frac{1}{\phi \text{の訪問回数 at }\widetilde{\Lambda}}\right)
\approx
(2\Lambda^{-1} - \widetilde{\Lambda}^{-1})\phi = \lambda \phi
$$

を満たします．ここで，

* $\widetilde{\Lambda}$では１度も訪れなかった$\phi$が$\Lambda$で訪問された場合，$左辺=(2 - \infty)\phi = -\infty\phi$になるので，固有値は$\lambda < 0$のはずです．
* $\widetilde{\Lambda}$で100回訪問された$\phi$が$\Lambda$でも訪問された場合，$左辺=(\frac{2}{101} - \frac{1}{100})\phi \approx \phi$になるので，固有値は$\lambda > 0$になります．

このように，$\phi$の訪問数が少ないほど，つまり新情報の驚きが大きいほど，$\Lambda^{-1} - \widetilde{\Lambda}^{-1}$の固有値は負に近づきます．Low-switchingはこれを利用しています．

---




## リグレット解析

基本的には普通のLinear MDPと同じなので，使える箇所は使いまわします．

---

高確率で次が成立：

$$
\begin{aligned}
& \forall(k, h) \in[K] \times[H]: \\
& \left\|\sum_{\tau=1}^{k-1} \phi_h^\tau\left[\widetilde{V}_{h+1}^k\left(x_{h+1}^\tau\right)-\mathbb{P}_h \widetilde{V}_{h+1}^k\left(x_h^\tau, a_h^\tau\right)\right]\right\|_{\left(\Lambda_h^k\right)^{-1}} \\
& \lessapprox d H
\end{aligned}
$$

---

任意の方策$\pi$について次が成立：

$$
\begin{aligned}
\left\langle\phi(x, a), \mathbf{w}_h^k\right\rangle-Q_h^\pi(x, a)= & \mathbb{P}_h\left(\widetilde{V}_{h+1}^k-V_{h+1}^\pi\right)(x, a) \\
& +\Delta_h^k(x, a)
\end{aligned}
$$

ここで，
$$
\left|\Delta_h^k(x, a)\right| \leq \beta \sqrt{\phi(x, a)^{\top}\left(\Lambda_h^k\right)^{-1} \phi(x, a)}
$$

---

Optimism：$\widetilde{Q}_h^k(x, a) \geq Q_h^{\star}(x, a)$

---

* $\widetilde{\delta}_h^k=\widetilde{V}_h^{\tilde{k}}\left(x_h^k\right)-V_h^{\pi_{\tilde{k}}}\left(x_h^k\right)$
    * 直感：「デプロイ$k$」で「前回デプロイされた$\widetilde{V}^{\widetilde{k}}$の$x^k$での値」vs「今の方策の価値」
* $\widetilde{\zeta}_{h+1}^k=\mathbb{E}\left[\widetilde{\delta}_{h+1}^k \mid x_h^k, a_h^k\right]-\widetilde{\delta}_{h+1}^k$
    * マルンゲール差分
 
とします．このとき，

$$
\widetilde{\delta}_h^k \leq \widetilde{\delta}_{h+1}^k+\widetilde{\zeta}_{h+1}^k+2 \beta \sqrt{\left(\phi_h^k\right)^{\top}\left(\Lambda_h^{\tilde{k}}\right)^{-1} \phi_h^k}
$$

---

**リグレットの証明**

Optimismから，

$$
\begin{aligned}
\operatorname{Regret}(K) & =\sum_{k=1}^K\left[V_1^*\left(x_1^k\right)-V_1^{\pi_k}\left(x_1^k\right)\right] \\
& \leq \sum_{k=1}^K\left[\widetilde{V}_1^{\tilde{k}}\left(x_1^k\right)-V_1^{\pi_{\tilde{k}}}\left(x_1^k\right)\right]
= \sum_{k=1}^K \widetilde{\delta}_1^k
\end{aligned}
$$
が成立します．

$\widetilde{\delta}$をバウンドしていきましょう．
まず，

$$
\left[\widetilde{Q}_h^{\tilde{k}}-Q_h^{\pi_{\tilde{k}}}\right](x, a)=\mathbb{P}_h\left(V_{h+1}^{\tilde{k}}-V_{h+1}^{\pi_{\tilde{k}}}\right)(x, a)+\Delta_h^{\tilde{k}}(x, a)
$$

が成立します．ここで，
$$
\mathbb{P}_h\left(V_{h+1}^{\tilde{k}}-V_{h+1}^{\pi_{\tilde{k}}}\right)(x, a)
= \mathbb{E}[\widetilde{\delta}_{h+1}^k \mid x_h^k, a_h^k]
$$
なので，定義から，
$$
\begin{aligned}
\left(\widetilde{Q}_h^{\tilde{k}}-Q_h^{\pi_{\tilde{k}}}\right)\left(x_h^k, a_h^k\right) 
& = \Delta_h^{\tilde{k}}(x^k_h, a^k_h)+\mathbb{E}[\widetilde{\delta}_{h+1}^k\mid x_h^k, a_h^k] \\
& = \Delta_h^{\tilde{k}}(x, a)+\widetilde{\zeta}_h^k+\widetilde{\delta}_{h+1}^k
\end{aligned}
$$

$\zeta$のところはHoeffdingでいい感じにバウンドできます．
$$
\sum_{k=1}^K \sum_{h=1}^H \widetilde{\zeta}_h^k \leq H \sqrt{T \iota} .
$$

また，Linear MDPでよく見る再帰的な分解を使えば
$$
\begin{aligned}
\operatorname{Regret}(K) & \leq \sum_{k=1}^K \widetilde{\delta}_1^k \\
& \leq \sum_{k=1}^K \sum_{h=1}^H \widetilde{\zeta}_h^k+3 \beta \sum_{k=1}^K \sum_{h=1}^H \sqrt{\phi_h^k\left(\Lambda_h^{\tilde{k}}\right)^{-1} \phi_h^k}
\end{aligned}
$$
に直せます．最後に$\phi_h^k\left(\Lambda_h^{\tilde{k}}\right)^{-1} \phi_h^k \leq 2 \phi_h^k\left(\Lambda_h^k\right)^{-1} \phi_h^k$を使うと

$$
\leq H \sqrt{T \iota}+6 H \sqrt{d K \iota}
$$

で抑えられます．
これで証明終わりです．



